# 영업 성공 여부 분류 경진대회

## 1. 데이터 확인

### 필수 라이브러리

In [1]:
import random
import os

import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold #교차검증
from sklearn.model_selection import GridSearchCV #하이퍼파라미터 튜닝을 위한 불러오기
from sklearn.preprocessing import MinMaxScaler   #스케일링

### 랜덤 시드 고정(학습 시마다 동일 결과)

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
my_seed = 400
seed_everything(my_seed) # Seed 고정

### 데이터 셋 읽어오기

In [3]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

## 2. 데이터 전처리

### 2-1 레이블 인코딩

In [4]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [5]:
# 레이블 인코딩할 칼럼들
label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_country.1",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

다시 학습 데이터와 제출 데이터를 분리합니다.

In [6]:
for col in label_columns:  
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]

### 2-2. 학습, 검증 데이터 분리

In [7]:
x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

### 2-3 결측치(NA) 처리

In [8]:
# train

train_nan = x_train.copy()

zero_cols = ['com_reg_ver_win_rate',
             'historical_existing_cnt',
             'id_strategic_ver',
             'it_strategic_ver',
             'idit_strategic_ver']

for col in zero_cols:
    train_nan[col] = train_nan[col].fillna(0)

mean_cols = ['ver_win_rate_x','ver_win_ratio_per_bu']
for col in mean_cols:
    train_nan[col] = train_nan[col].fillna(train_nan[col].mean())
    
x_train = train_nan

# validation

val_nan = x_val.copy()

val_zero_cols = ['com_reg_ver_win_rate',
                 'historical_existing_cnt',
                 'id_strategic_ver',
                 'it_strategic_ver',
                 'idit_strategic_ver']

for col in val_zero_cols:
    val_nan[col] = val_nan[col].fillna(0)

val_mean_cols = ['ver_win_rate_x','ver_win_ratio_per_bu']
for col in val_mean_cols:
    val_nan[col] = val_nan[col].fillna(train_nan[col].mean())

x_val = val_nan

# test

test_nan = df_test.copy()

test_zero_cols = ['com_reg_ver_win_rate',
                  'historical_existing_cnt',
                  'id_strategic_ver',
                  'it_strategic_ver',
                  'idit_strategic_ver']

for col in test_zero_cols:
    test_nan[col] = test_nan[col].fillna(0)
    
test_mean_cols = ['ver_win_rate_x','ver_win_ratio_per_bu']
for col in test_mean_cols:
    test_nan[col] = test_nan[col].fillna(train_nan[col].mean())

df_test = test_nan

### 2-4 상관계수 높고 중요도 낮은 Feature 삭제

In [9]:
x_train = x_train.drop(['ver_win_rate_x','ver_win_ratio_per_bu'],axis=1)
x_val = x_val.drop(['ver_win_rate_x','ver_win_ratio_per_bu'],axis=1)
df_test = df_test.drop(['ver_win_rate_x','ver_win_ratio_per_bu'],axis=1)

## 3. 모델 학습

### 모델 정의 

In [10]:
model = DecisionTreeClassifier()


### 모델 학습

In [11]:
model.fit(x_train, y_train)

DecisionTreeClassifier()

### 모델 성능 보기

In [12]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [13]:
pred = model.predict(x_val.fillna(0))
get_clf_eval(y_val, pred)

오차행렬:
 [[  736   211]
 [  254 10659]]

정확도: 0.9608
정밀도: 0.7434
재현율: 0.7772
F1: 0.7599


## 4. 제출하기

### 테스트 데이터 예측

In [14]:
# 예측에 필요한 데이터 분리
x_test = df_test.drop(["is_converted", "id"], axis=1)

In [15]:
test_pred = model.predict(x_test.fillna(0))
sum(test_pred) # True로 예측된 개수

1177

### 제출 파일 작성

In [16]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**